In [0]:
import pandas as pd # 데이터 전처리
import numpy as np # 데이터 전처리
import matplotlib.pyplot as plt # 데이터 시각화
import itertools
from datetime import datetime, timedelta # 시간 데이터 처리
from statsmodels.tsa.arima_model import ARIMA # ARIMA 모델
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm

from time import time
import datetime


In [0]:
best = pd.read_csv("./일요일날씨변수추가SARIMAX_HOUR_33점55점파일의day와month_30점01.csv")

test = pd.read_csv("./test_lgbm_interpol_1119.csv")
test['time'] = pd.to_datetime(test['time']) 

In [0]:
test['date'] = test['time'].dt.weekday_name

In [0]:
test_sun = test[test['date']=='Sunday'].reset_index().drop('index',axis=1)

In [0]:
# 상가로 추측되는 여름 데이터
kk_c1= ['time','X125', 'X76', 'X106', 'X94', 'X123', 'X111', 'X73', 'X67', 'X91', 'X87', 'X126', 'X120', 'X110', 'X105', 'X88', 'X116', 'X68', 'X124', 'X82', 'X80', 'X102', 'X70', 'X75', 'X107']
# 상가로 추측되는 봄 데이터
kk_c2= ['time','X26', 'X16', 'X7', 'X18', 'X41', 'X55', 'X5', 'X43', 'X59', 'X10', 'X62', 'X61', 'X32', 'X31', 'X30', 'X51', 'X35', 'X44', 'X37', 'X42', 'X50', 'X47', 'X20']
# 아파트로 추측되는 데이터
kk_n = ['time','X303', 'X241', 'X435', 'X402', 'X352', 'X305', 'X350', 'X326', 'X299', 'X160', 'X211', 'X349', 'X407', 'X330', 'X132', 'X231', 'X225', 'X286', 'X354', 'X353', 'X255', 'X393', 'X157', 'X323', 'X130', 'X269', 'X172', 'X289', 'X216', 'X398', 'X233', 'X214', 'X270', 'X371', 'X200', 'X163', 'X364', 'X378', 'X271', 'X462', 'X427', 'X263', 'X450', 'X206', 'X133', 'X195', 'X134', 'X297', 'X175', 'X254', 'X196', 'X412', 'X337', 'X406', 'X414', 'X260', 'X252', 'X455', 'X320', 'X276', 'X423', 'X218', 'X399', 'X213', 'X379', 'X468', 'X253', 'X247', 'X262', 'X248', 'X357', 'X215', 'X177', 'X228', 'X187', 'X245', 'X242', 'X376', 'X166', 'X472', 'X275', 'X313', 'X201', 'X408', 'X419', 'X460', 'X367', 'X143', 'X312', 'X410', 'X389', 'X300', 'X470', 'X445', 'X158', 'X421', 'X178', 'X308', 'X217', 'X229', 'X358', 'X148', 'X397', 'X363', 'X165', 'X296', 'X219', 'X431', 'X302', 'X314', 'X405', 'X315', 'X176', 'X396', 'X449', 'X383', 'X473', 'X205', 'X161', 'X403', 'X341', 'X149', 'X377', 'X232', 'X333', 'X140', 'X422', 'X386', 'X250', 'X142', 'X235', 'X446', 'X331', 'X186', 'X243', 'X194', 'X183', 'X139', 'X459', 'X457', 'X368', 'X282', 'X426', 'X192', 'X283', 'X329', 'X223', 'X266', 'X443', 'X347', 'X230', 'X155','X481']

## 세 그룹으로 나누기

In [0]:
# 연속된 값 있는 부분 집단별로 제거
test_sun1 = pd.concat([test_sun[kk_c1].iloc[:216], test_sun[kk_c1].iloc[504:]]).reset_index().drop('index',axis=1)
test_sun2 = test_sun[kk_c2].iloc[1000:]
test_sun3 = test_sun[kk_n]

In [0]:
test_sun1 = test_sun1.set_index('time')
test_sun2 = test_sun2.set_index('time')
test_sun3 = test_sun3.set_index('time')

# 날씨데이터

In [0]:
w_hour = pd.read_excel('../기상정보/인천_시간별__기상자료(16-18)_축소__7월1일.xlsx')
w_hour['일시'] = pd.to_datetime(w_hour['일시'])
w_hour = w_hour.rename(columns={'기온(°C)':'temp','습도(%)':'humid','강수량(mm)':'preci','전운량(10분위)':'cloud','일시':'time', '풍속(m/s)': 'wind'})
w_hour = w_hour[['time','temp','humid','cloud','preci','wind']]
w_hour['cloud'] = w_hour['cloud'].interpolate(method='linear')
w_hour['temp'] = w_hour['temp'].interpolate(method='linear')
w_hour['humid'] = w_hour['humid'].interpolate(method='linear')
w_hour['preci'] = w_hour['preci'].fillna(0)

# 준비

In [0]:
def new_df(data,weather):
    place_id=[]; time=[] ; target=[] # 빈 리스트를 생성합니다.
    for i in data.columns:
        for j in range(len(data)):
            place_id.append(i) # place_id에 미터 ID를 정리합니다.
            time.append(data.index[j]) # time에 시간대를 정리합니다.
            target.append(data[i].iloc[j]) # target에 전력량을 정리합니다.

    new_df=pd.DataFrame({'place_id':place_id,'time':time,'target':target})
    new_df=new_df.dropna() # 결측치를 제거합니다.
    new_df = pd.merge(new_df,weather, on='time', how='left')
    new_df=new_df.set_index('time') # time을 인덱스로 저장합니다.
    
    return new_df

In [0]:
sun1 = new_df(test_sun1,w_hour)
sun2 = new_df(test_sun2,w_hour)
sun3 = new_df(test_sun3,w_hour)

In [0]:
sun1[sun1.columns[1:]].corr()['target']

target    1.000000
temp      0.014008
humid    -0.049756
cloud    -0.017393
preci    -0.007042
wind      0.049605
Name: target, dtype: float64

# 파라미터

In [0]:
p = q = range(0, 2)
d =  range(0, 2)
#s = range(24,25)
pdq1 = list(itertools.product(p, d, q))

p1 = q1 = range(0, 2)
d1 =  range(0, 2)
s1 = range(24,25)
pdq = list(itertools.product(p1, d1, q1,s1))

In [0]:
def get_optimal_params2(y, exog):

    param_dict = {}
    for param1 in pdq1:
        for param in pdq:
            try:
                model = SARIMAX(y, order=param1, seasonal_order=param,exog=exog)
                results_SARIMAX = model.fit(disp=-1)
                param_dict[results_SARIMAX.aic] = [param, param1]
            except:
                continue

    min_aic = min(param_dict.keys())
    optimal_params = param_dict[min_aic]
    return optimal_params, min_aic, param_dict

# 예측

In [0]:
submission = pd.read_csv("../submission.csv")

In [0]:
import gc
gc.collect()

14

그룹2

In [0]:
from time import time
import datetime
from sklearn.metrics import mean_squared_error

total_start_time = time()
agg={}
total_start_time = time()
for key in sun2['place_id'].unique(): # 미터ID 200개의 리스트를 unique()함수를 통해 추출합니다.
    temp = sun2.loc[sun2['place_id']==key] # 미터ID 하나를 할당합니다.
    #temp_1h=temp.resample('1h').sum() # 1시간 단위로 정리합니다.
    #temp_1day=temp.resample('D').sum() # 1일 단위로 정리합니다.
    start_time = time()
    
    # 시간별 예측
    param, min_aic, param_dict = get_optimal_params2(temp.reset_index()['target'], 
                               temp.reset_index()[['temp', 'humid', 'wind']])
    
    model = SARIMAX(temp.reset_index()['target'], order=param[1], seasonal_order=param[0],
                    exog=temp.reset_index()[['temp', 'humid', 'cloud']])
    
    results_SARIMAX = model.fit(maxiter=1000, disp=-1, exog=temp.reset_index()[['temp', 'humid', 'wind']])
    
    fcst = results_SARIMAX.forecast(24, exog = w_hour[['temp', 'humid', 'wind']].iloc[-192:-168])
    
    a = pd.DataFrame() # a라는 데이터프레임에 예측값을 정리합니다.
    
    for i in range(24):
        a['X2018_7_1_'+str(i+1)+'h']=[fcst.iloc[i]] # column명을 submission 형태에 맞게 지정합니다.
        a['X2018_7_1_'+str(i+1)+'h'] = np.where(a['X2018_7_1_'+str(i+1)+'h']<0, temp['target'].min(), a['X2018_7_1_'+str(i+1)+'h'])
        
    # 그래프
    plt.plot(range(0,24),a[a.columns[0:24]].iloc[0], label='pred')
    plt.plot(range(0,24),temp['target'].iloc[-24:],label='last week')
    plt.plot(range(0,24),best[best['meter_id']==col].iloc[0].iloc[1:25], label='30.1 pred')
    plt.legend()
    plt.title(key)
    plt.show()
    for i in range(10):
        a['X2018_7_'+str(i+1)+'_d']=0 # column명을 submission 형태에 맞게 지정합니다.

    # 월별 예측
    # 일별로 예측하여 7월 ~ 11월의 일 수에 맞게 나누어 합산합니다.
    #fcst = results_ARIMA.forecast(153)
    #a['X2018_7_m'] = [np.sum(fcst['target'][:31])] # 7월 
    #a['X2018_8_m'] = [np.sum(fcst['target'][31:62])] # 8월
    #a['X2018_9_m'] = [np.sum(fcst['target'][62:92])] # 9월
    #a['X2018_10_m'] = [np.sum(fcst['target'][92:123])] # 10월
    #a['X2018_11_m'] = [np.sum(fcst['target'][123:153])] # 11월
    a['X2018_7_m'] = 0 # 7월 
    a['X2018_8_m'] = 0 # 8월
    a['X2018_9_m'] = 0 # 9월
    a['X2018_10_m'] = 0 # 10월
    a['X2018_11_m'] = 0 # 11월
    a['meter_id'] = key 
    agg[key] = a[submission.columns.tolist()]

    print(key)
    print('param, aic, SMAPE : ',param, ', ',min_aic)
    print('finished in {}'.format(str(datetime.timedelta(seconds=time() - start_time))))
print('complited in {}'.format(str(datetime.timedelta(seconds=time() - total_start_time))))
print('---- Modeling Done ----')

KeyError: "['wind'] not in index"

In [0]:
SARIMAX_21 = pd.concat(agg, ignore_index=False)
SARIMAX_22 = SARIMAX_21.reset_index().drop(['level_0','level_1'], axis=1)
SARIMAX_22['id'] = SARIMAX_22['meter_id'].str.replace('X','').astype(int)
SARIMAX_22 =  SARIMAX_22.sort_values(by='id', ascending=True).drop(['id'], axis=1).reset_index(drop=True)

# 제출

In [0]:
SARIMAX_22.to_csv('lgbm_sarimax0701_c2.csv',index=False)